In [ ]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath(".."))

In [ ]:
import os
from av2.datasets.sensor.av2_sensor_dataloader import AV2SensorDataLoader
from pathlib import Path
from av2.structures.sweep import Sweep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from config import CONFIG
from prototype_utils import plot_point_cloud_and_bboxes,  bboxes_df_to_numpy_corners, filter_roi

In [ ]:
home = os.path.join(os.path.expanduser('~'), CONFIG['HOME_PATH'][CONFIG['OS']])

dataset_path = Path(os.path.join(home, "dataset", "av2", "train"))

av2 = AV2SensorDataLoader(data_dir=dataset_path, labels_dir=dataset_path)

scene_idx = 2
frame_idx = 10

scene_id = av2.get_log_ids()[scene_idx]
frame_id = av2.get_ordered_log_lidar_timestamps(scene_id)[frame_idx]
frame_path = av2.get_lidar_fpath_at_lidar_timestamp(scene_id, frame_id)

points = Sweep.from_feather(frame_path).xyz

In [ ]:
ps_path = os.path.join(home, *CONFIG['BBOX_FILE_PATHS']['ROI'])

scene_id = os.listdir(ps_path)[scene_idx]
scene_path = os.path.join(ps_path, scene_id)

frame_id = os.listdir(scene_path)[frame_idx]
frame_path = os.path.join(ps_path, scene_id, frame_id)

In [ ]:
scene_id, frame_id

In [ ]:
bboxes_df = pd.read_feather(frame_path)
bboxes_df.head()

In [ ]:
bboxes_np = bboxes_df_to_numpy_corners(bboxes_df)

In [ ]:
points_roi = filter_roi(points)

In [ ]:
plot_point_cloud_and_bboxes(points_roi[:,:2], bboxes_np, title="BEV Point Cloud")

In [ ]:
bboxes_df['aspect_ratio'] = bboxes_df['box_width'] / bboxes_df['box_length']
bboxes_df['aspect_ratio']

bboxes_df['area'] = bboxes_df['box_width'] * bboxes_df['box_length']

In [ ]:
bboxes_df.head()

In [ ]:
bboxes_df.hist(column='aspect_ratio', bins=50, range=(0, 1))

In [ ]:
def plot_aspect_ratio_vs_area(df, aspect_ratio_col, area_col, title="Scatter Plot of Aspect Ratio vs Area"):
    """
    Plots a scatter plot of aspect ratio vs area.

    Args:
        df (pd.DataFrame): The DataFrame containing aspect ratio and area.
        aspect_ratio_col (str): The column name for aspect ratios in the DataFrame.
        area_col (str): The column name for areas in the DataFrame.
        title (str): The title of the scatter plot.
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(df[aspect_ratio_col], df[area_col], alpha=0.7, edgecolor='k')
    plt.title(title, fontsize=14)
    plt.xlabel('Aspect Ratio', fontsize=12)
    plt.ylabel('Area', fontsize=12)
    plt.grid(True)
    plt.show()

# Example usage:
plot_aspect_ratio_vs_area(bboxes_df, 'aspect_ratio', 'area')


In [ ]:
from stats_filter2 import apply_rect_filter, apply_large_sq_filter

config_rect_filter = CONFIG['RECT_FILTER_THRESHOLDS']['ROI']['IOU_THRESHOLD_0.3']
config_sq_filter = CONFIG['SQUARE_FILTER_THRESHOLDS']['ROI']['IOU_THRESHOLD_0.3']

rect_filter_max_ratio = config_rect_filter['MAX_RATIO']
rect_filter_max_area = config_rect_filter['MAX_AREA']

sq_filter_min_ratio = config_sq_filter['MIN_RATIO']
sq_filter_min_area = config_sq_filter['MIN_AREA']

_, rect_filter_df = apply_rect_filter(bboxes_df, 'aspect_ratio', 'area',rect_filter_max_ratio,rect_filter_max_area )

_ , square_filter_df = apply_large_sq_filter(bboxes_df, 'aspect_ratio', 'area', sq_filter_min_ratio, sq_filter_min_area)

_ , combined_df = apply_large_sq_filter(rect_filter_df, 'aspect_ratio', 'area', sq_filter_min_ratio, sq_filter_min_area)

In [ ]:
plot_point_cloud_and_bboxes(points_roi[:,:2],
                            bboxes_df_to_numpy_corners(rect_filter_df),
                            title="Rects_filter applied")

In [ ]:
plot_point_cloud_and_bboxes(points_roi[:,:2],
                            bboxes_df_to_numpy_corners(square_filter_df),
                            title="Square Filter")

In [ ]:
plot_point_cloud_and_bboxes(points_roi[:,:2],
                            bboxes_df_to_numpy_corners(combined_df),
                            title="Combined Filter")